<a href="https://colab.research.google.com/github/wallik2/JobsDB-WebScraper/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⛺

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd, numpy as np
import re

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
Job_keyword = 'data'

In [ ]:
# Fah directory
try:
  df = pd.read_csv(f'/content/drive/MyDrive/keyword: {Job_keyword}.csv')

# Run directory
except:
  df = pd.read_csv(f'/content/drive/MyDrive/Jobsdb/keyword: {Job_keyword}.csv',index_col=0)

In [ ]:
# merge

df['skill requirement'] = df['skill requirement 1'] + df['skill requirement 2'] + df['skill requirement 3']

In [ ]:
df.head()

,job title,more info,job_company,job_location,skill requirement 1,skill requirement 2,skill requirement 3,posted time,skill requirement
0,Data Analyst,https://th.jobsdb.com//th/en/job/data-analyst-...,Siam Piwat Group,Pathumwan,Deliver in-depth analysis,Performance tracking,End-to-end data analytics lifecycle,14h ago,Deliver in-depth analysisPerformance trackingE...
1,"Fresh Graduate, Data Scientist",https://th.jobsdb.com//th/en/job/fresh-graduat...,"Siam Commercial Bank Public Co., Ltd. (SCB)",Jatuchak,"statistical programming (e.g., Python, R,pandas)","database software (e.g., SQL, PySpark)","hypothesis testing, regression, machine learning",18h ago,"statistical programming (e.g., Python, R,pand..."
2,Data Analyst,https://th.jobsdb.com//th/en/job/data-analyst-...,"Ngernturbo Co., Ltd./ บริษัท เงินเทอร์โบ จำกัด",Prakket,Challenging work.,Competitive package.,Vibrant culture.,14h ago,Challenging work.Competitive package.Vibrant c...
3,Data Analyst,https://th.jobsdb.com//th/en/job/data-analyst-...,"Tri Petch IT Solutions Co., Ltd.",Jatuchak,5 Day-Work / Bonus twice a year / MRT Phahonyo...,Health Insurance/ Flextime/ Excellent Benefits...,Training / Gym / BTS Phahonyothin 24 /Contribu...,14h ago,5 Day-Work / Bonus twice a year / MRT Phahonyo...
4,Data Analytics Officer,https://th.jobsdb.com//th/en/job/data-analytic...,"Predictive Co., Ltd.",Wattana,NaN,NaN,NaN,18h ago,NaN


In [ ]:
# We don't use them anymore
df.drop(columns= ['skill requirement 1', 'skill requirement 2','skill requirement 3'],inplace=True)


# Drop N/A 
df.dropna(inplace=True)

In [ ]:
# all lower case

df['skill requirement'] = df['skill requirement'].str.lower()

In [ ]:
df.reset_index(drop=True, inplace=True)



---



# ⛄

Stop words like A, can, be, I are mostly noise 

![alt text](https://media.geeksforgeeks.org/wp-content/cdn-uploads/Stop-word-removal-using-NLTK.png)

In [ ]:
import nltk

#removing stopwords 
from nltk.corpus import stopwords

#normalize word to base form
from nltk.stem import WordNetLemmatizer
#Text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Similarity between text
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
skill_req = df['skill requirement']

#removing stopwords

stop = stopwords.words('english')
only_text = skill_req.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
only_text.head()

0    deliver in-depth analysisperformance trackinge...
1    statistical programming (e.g., python, r,panda...
2    challenging work.competitive package.vibrant c...
3    5 day-work / bonus twice year / mrt phahonyoth...
4    data analystcrm platforms, campaign management...
Name: skill requirement, dtype: object

In [ ]:
only_text = only_text.apply(lambda x : filter(None,x.split(" ")))
print(only_text.head())

0    <filter object at 0x7fc658453990>
1    <filter object at 0x7fc658453cd0>
2    <filter object at 0x7fc658453e10>
3    <filter object at 0x7fc658457290>
4    <filter object at 0x7fc658457510>
Name: skill requirement, dtype: object


In [ ]:
# (eaten, eating, ate ) -> Eat
lemmatizer = WordNetLemmatizer();lemmatizer

<WordNetLemmatizer>

In [ ]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:

only_text = only_text.apply(lambda x : [lemmatize_words(y) for y in x])
print(only_text.head())

0    [deliver, in-depth, analysisperformance, track...
1    [statistical, programming, (e.g.,, python,, r,...
2    [challenging, work.competitive, package.vibran...
3    [5, day-work, /, bonus, twice, year, /, mrt, p...
4    [data, analystcrm, platforms,, campaign, manag...
Name: skill requirement, dtype: object


In [ ]:
only_text = only_text.apply(lambda x : " ".join(x))
print(only_text.head())

0    deliver in-depth analysisperformance trackinge...
1    statistical programming (e.g., python, r,panda...
2    challenging work.competitive package.vibrant c...
3    5 day-work / bonus twice year / mrt phahonyoth...
4    data analystcrm platforms, campaign management...
Name: skill requirement, dtype: object




---



# ✌

In [ ]:
# ID
Applicant_id =  15#@param {type:"integer"}

# QUALITY (Your skill)
Quality = "English Critical thinking and problem solving. Teamwork and collaboration. Professionalism and strong work ethic. Oral and written communications skills. Leadership." #@param {type:"string"}

In [ ]:
data ={'Applicant_id':[Applicant_id],
       'Quality':[Quality] }

       
Applicant = pd.DataFrame(data)

In [ ]:
Quality = [Quality]

In [ ]:
def vecIt(corpus):
  tfidf_vectorizer = TfidfVectorizer()

  #fitting and transforming the vector
  tfidf_jobid = tfidf_vectorizer.fit_transform((corpus)) 

  #creating tf-idf of user query and computing cosine similarity of user with job
  user_tfidf = tfidf_vectorizer.transform(Quality)
  output = list(map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid))

  # First best 50 (highest similarity score)
  winners = sorted(range(len(output)), key=lambda i: output[i], reverse=True)[:50]
  
  score = output
  
  return winners, score

In [ ]:
winner = vecIt(only_text)[0]
score = vecIt(only_text)[1]

# Easier to interpret
round_score = np.round(score,decimals=5)

In [ ]:
recommendation = pd.DataFrame(columns = ['ApplicantID', 'Job'])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

Range of Similarity score : [0,1]


In [ ]:
for index, k in enumerate(winner,start=1):

  if index == 1: print(f'🥇 Rank {index}.' ,df.loc[k,'job title'],'\n')
  elif index == 2: print(f'🥈 Rank {index}.' ,df.loc[k,'job title'],'\n')
  elif index == 3: print(f'🥉 Rank {index}.' ,df.loc[k,'job title'],'\n')
  else: print(f'Rank {index}.' ,df.loc[k,'job title'],'\n')

  print(f'Similarity score :',flatten(round_score[k])[0]*100,'%')
  print(f'Company:' ,df.loc[k,'job_company'])
  print(f'More information :',df.loc[k,'more info'])


  print("\n"*2,"#"*120,"\n"*2)

     #print(f'The job is {df.loc[k,'job title']} The company is {df['job_company'][k]} \n{df['more info'][k]} The similar score is ,{output2 [k]}')

🥇 Rank 1. Solution Architect Expert 

Similarity score : 25.517 %
Company: True Corporation Public Company Limited
More information : https://th.jobsdb.com//th/en/job/solution-architect-expert-300003002431778?token=0~aa298e36-245b-490d-86c8-95e7eb6262a5&sectionRank=2260&jobId=jobsdb-th-job-300003002431778


 ######################################################################################################################## 


🥈 Rank 2. Manager – Strategy & Business Design (Monitor Deloitte) – Financial Services - TH 

Similarity score : 24.634 %
Company: Deloitte Consulting Ltd.
More information : https://th.jobsdb.com//th/en/job/manager-strategy-business-design-monitor-deloitte-financial-services-th-300003002439967?token=0~31d9af01-f0ab-46ce-a8ae-8ce973beed21&sectionRank=4134&jobId=jobsdb-th-job-300003002439967


 ######################################################################################################################## 


🥉 Rank 3. Service Engineer (PLC Trainer) / Up